# Maintain symlinks

The data is split over two disks: `/data` and `/psr_archive`. The former is a locally-mounted RAID array but isn't particularly backed up. The latter is an NFS-mounted archival disk with snapshotting and RAID, so, permanent storage for data that won't ever change. The intent is that raw data and raw data rearranged into observation directories should live on `/psr_archive` but that everything else should live on `/data`. Doing this without the scripts knowing about it requires a suitable arrangement of symlinks. This notebook is intended to allow maintenance of these symlinks, ensuring that the right data is on

In [2]:
import os
from glob import glob
from os.path import join
import shutil

In [34]:
permanent_base = "/psr_archive/hessels/archibald/0337+17/"
temporary_base = "/data/archibald/0337+1715/"

In [51]:
def mywalk(base_dir, extra_dir=""):
    d = join(base_dir,extra_dir)
    dirs = []
    files = []
    for e in os.listdir(d):
        if os.path.isdir(join(d,e)):
            dirs.append(e)
        else:
            files.append(e)
    yield base_dir, extra_dir, dirs, files
    for d in dirs:
        for b, e, di, f in mywalk(base_dir, join(extra_dir,d)):
            yield b, e, di, f

In [30]:
move_and_link = []
for root, dirs, files in os.walk(join(temporary_base,"cal")):
    subp = root[len(temporary_base):]
    for f in files:
        tf = os.path.join(root, f)
        pf = os.path.join(permanent_base, subp, f)
        if (should_be_permanent(os.path.join(subp,f))
            and not os.path.islink(tf)):
            move_and_link.append((tf,pf))
            print "move and link:", tf, pf
# FIXME: make cals permanent
for f in sorted(glob(join(temporary_base,"obs","*","*"))):
    subp = f[len(temporary_base):]
    pf = join(permanent_base,subp)
    if not os.path.isdir(f) and not os.path.islink(f):
        print f, pf
        move_and_link.append((f,pf))

In [15]:
for (tf,pf) in move_and_link:
    if os.path.exists(pf):
        raise ValueError("Can't move %s to %s: destination exists!" % (tf,pf))
    d = os.path.split(pf)[0]
    if not os.path.exists(d):
        os.makedirs(d)
    shutil.move(tf, pf)
    os.symlink(pf, tf)

In [17]:
to_link = []
for root, dirs, files in os.walk(permanent_base):
    subp = root[len(permanent_base):]
    for f in files:
        tf = os.path.join(temporary_base, subp, f)
        pf = os.path.join(permanent_base, subp, f)
        if not os.path.exists(tf):
            print "link:", pf, tf
            to_link.append((pf,tf))

link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56203.26_GBT_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56203.26_GBT_1400.pdf
link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56063.64_WSRT_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56063.64_WSRT_1400.pdf
link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56444.69_GBT_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56444.69_GBT_1400.pdf
link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56034.89_GBT_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56034.89_GBT_1400.pdf
link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56005.83_AO_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56005.83_AO_1400.pdf
link: /psr_archive/hessels/archibald/0337+17/summary/subband/normal/56032.59_WSRT_1400.pdf /data/archibald/0337+1715/summary/subband/normal/56032.59_WSRT_1400.pdf
link: /psr_archive/hessels/archi

In [26]:
for (pf,tf) in to_link:
    if os.path.islink(pf) and not os.path.exists(pf):
        os.unlink(pf)
    if os.path.islink(tf) and not os.path.exists(tf):
        os.unlink(tf)
for (pf,tf) in to_link:
    if not os.path.exists(pf):
        continue
    d = os.path.basename(tf)[0]
    if not os.path.exists(d):
        os.makedirs(d)
    if not os.path.exists(tf):
        print pf, tf
        os.symlink(pf,tf)

/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57782_0337+17_0002_0001.fits.success /data/archibald/0337+1715/raw/GBT/new/guppi_57782_0337+17_0002_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57720_0337+17_0002_0001.fits.success /data/archibald/0337+1715/raw/GBT/new/guppi_57720_0337+17_0002_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57826_0337+17_0004_0001.fits.success /data/archibald/0337+1715/raw/GBT/new/guppi_57826_0337+17_0004_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57577_0337+17_0002_0001.fits.success /data/archibald/0337+1715/raw/GBT/new/guppi_57577_0337+17_0002_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57697_0337+17_0002_0001.fits.success /data/archibald/0337+1715/raw/GBT/new/guppi_57697_0337+17_0002_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/GBT/new/guppi_57878_0337+17_0002_0007.fits /data/archibald/0337+1715/raw/GBT/new/g

OSError: [Errno 2] No such file or directory

In [27]:
!ls -l /psr_archive/hessels/archibald/0337+17/bs/57705.19_AO_1400/raw_0001.ar

-rw-r--r-- 1 archibald 9000 471072960 May 24 23:19 /psr_archive/hessels/archibald/0337+17/bs/57705.19_AO_1400/raw_0001.ar


In [28]:
!ls -l /data/archibald/0337+1715/bs/57705.19_AO_1400/raw_0001.ar

ls: cannot access /data/archibald/0337+1715/bs/57705.19_AO_1400/raw_0001.ar: No such file or directory


In [25]:
!ls -l /misc/astron/archibald/projects/triplesystem/processing/data/obs/56203.26_GBT_1400/subband/normal/summary.pdf

ls: cannot access /misc/astron/archibald/projects/triplesystem/processing/data/obs/56203.26_GBT_1400/subband/normal/summary.pdf: No such file or directory


## Update mirroring

In [54]:
for root, sub, dirs, files in mywalk(permanent_base):
    if sub == "":
        del dirs[dirs.index("raw")]
    for f in files:
        pf = join(root, sub, f)
        d = join(temporary_base, sub)
        tf = join(d, f)
        if not os.path.exists(tf):
            if not os.path.exists(d):
                os.makedirs(d)
            os.symlink(pf, tf)

## Clean up raw data



In [22]:
# raw data should exist only on psr_archive
# the raw data directory should be symlinked
raw_temporary = "/data/archibald/0337+1715/raw-aside"
raw_permanent = "/psr_archive/hessels/archibald/0337+17/raw"

nonlinks = []
if os.path.islink(raw_temporary):
    raise ValueError
for root, dirs, files in os.walk(raw_temporary):
    for f in files:
        if not os.path.islink(f):
            nonlinks.append(join(root,f))

In [24]:
exist_both = []
to_move = []
for tf in nonlinks:
    tfs = tf.split("/")
    rts = raw_temporary.split("/")
    pf = join(raw_permanent, *(tfs[len(rts):]))
    if os.path.exists(pf):
        exist_both.append((tf, pf))
    else:
        to_move.append((tf,pf))
    #print tf, pf

In [25]:
len(to_move)

0

In [26]:
len(exist_both)

11

In [27]:
perm_links = []
temp_links = []
different = []
same_size = []
for tf, pf in exist_both:
    if os.path.islink(tf):
        temp_links.append((tf,pf))
    elif os.path.islink(pf):
        perm_links.append((tf,pf))
    elif os.path.getsize(tf)!=os.path.getsize(pf):
        different.append((tf,pf))
    else:
        same_size.append((tf,pf))

In [28]:
len(perm_links)

0

In [29]:
len(temp_links)

11

In [30]:
len(different)

0

In [31]:
len(same_size)

0

In [32]:
for tf, pf in temp_links:
    if not os.path.islink(tf):
        raise ValueError
    os.unlink(tf)
    b = os.path.dirname(tf)
    try:
        while True:
            os.rmdir(b)
            b = os.path.dirname(b)
    except OSError:
        pass

## Clear up danglinks

In [43]:
# walk doesn't follow links so this won't get the raw part
for root, dirs, files in os.walk(temporary_base):
    for sf in os.listdir(root):
        f = join(root,sf)
        if os.path.islink(f) and not os.path.exists(f):
            os.unlink(f)

In [39]:
f = "/data/archibald/0337+1715/obs/57236.41_GBT_1400/raw_0000.ar"
os.path.exists(f)

False

In [40]:
os.listdir("/data/archibald/0337+1715/obs/57236.41_GBT_1400")

['meta.pickle',
 'raw_0000.ar',
 'raw_0001.ar',
 'raw_0002.ar',
 'raw_0003.ar',
 'raw_0004.ar',
 'raw_0005.ar',
 'raw_0006.ar',
 'raw_0007.ar',
 'raw_0008.ar',
 'raw_0009.ar',
 'raw_0010.ar']

In [42]:
root = "/data/archibald/0337+1715/obs/57236.41_GBT_1400"
for f in os.listdir(root):
    print f
    if os.path.islink(f) and not os.path.exists(f):
        print "zap"
        os.unlink(f)


meta.pickle
raw_0000.ar
raw_0001.ar
raw_0002.ar
raw_0003.ar
raw_0004.ar
raw_0005.ar
raw_0006.ar
raw_0007.ar
raw_0008.ar
raw_0009.ar
raw_0010.ar
